In [1]:
import pickle,time
import cv2
import pyttsx3   #offline lib for tts
import serial  #for serial communication with arduino


In [2]:
!pip3 install opencv-contrib-python --upgrade

In [3]:
face_cascade=cv2.CascadeClassifier('C:\\Users\\HP\\anaconda3\\envs\\machinevision\\Lib\\site-packages\\cv2\\data\\haarcascade_frontalface_alt2.xml')
recognizer = cv2.face.LBPHFaceRecognizer_create()
recognizer.read('trainedModel.yml')  # Reading the stored trained model.

In [4]:
#Initiallize speech engine
engine = pyttsx3.init() 

#Loading the label-id relation to get the name of the labels.
with open("label_ids.pickle",'rb') as fr:    
    og_labels=pickle.load(fr)

#labels are of the form {name:id} we want to invert this form to {id:name}
labels={k:v for v,k in og_labels.items()}


In [5]:
print(labels)
# face_cascade=cv2.CascadeClassifier('haarcascade_profileface.xml')


{0: 'jamila', 1: 'luai'}


In [ ]:
def speak(text):  #fn to convert text to speech
    engine.say(text)
    engine.runAndWait() 

def face_recognition():
    flag_face_recognised=False   #to keep track if the user face is recognized
    flag_face_not_recognised=False

    no_of_adjacent_prediction=0
    no_face_detected=0  #to track the number of times the face is detected
    prev_predicted_name=''   #to keep track of the previously predicted face(w.r.t frame)
    cap=cv2.VideoCapture(0)
    count_frames = total_no_face_detected = 0





    while True:
        count_frames+=1
        ret,frame = cap.read()
        # print(ret,frame)
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        faces=face_cascade.detectMultiScale(gray,scaleFactor=1.2,minNeighbors=5)

        # print("FACES : ",faces)
        # print(type(frame))   #The frames are automatically converted into numpy arrays of pixels.
        for (x,y,w,h) in faces:

            total_no_face_detected+=1
            no_face_detected+=1

            # print(x,y,w,h)
            roi_gray=gray[y:y+h,x:x+w]  #roi(region of interest)
            # roi_color=frame[y:y+h,x:x+w]
            id,confidence=recognizer.predict(roi_gray)
            print(id,confidence)
            if(confidence>70):
                print("PREDICTED : ",labels[id])
                font=cv2.FONT_HERSHEY_SIMPLEX
                name=labels[id]
                clr=(255,255,255);thickness=2
                cv2.putText(frame,name.replace('_',' ').title(),(x,y-5),font,0.8,clr,1,cv2.LINE_AA)

            
                if(prev_predicted_name==name):
                    no_of_adjacent_prediction+=1
                else:
                    no_of_adjacent_prediction=0

                prev_predicted_name=name        



            # color=(255,0,0) #BGR 0-255
            thickness=1
            end_coord_x=x+w
            end_coord_y=y+h


            if(no_of_adjacent_prediction>15):   #no_of_adjacent_prediction is only updated when the confidence of classification is >80
                cv2.putText(frame,"Welcome home "+name.replace('_',' ').title(),(160,460),font,0.8,clr,thickness,cv2.LINE_AA)  #to print on frame
                flag_face_recognised=True
                no_of_adjacent_prediction=0
                no_face_detected=0

            elif(no_face_detected>=30):
                cv2.putText(frame,"Face Not Recognised",(160,460),font,0.8,clr,thickness,cv2.LINE_AA)
                flag_face_not_recognised=True
                no_of_adjacent_prediction=0
                no_face_detected=0

            cv2.rectangle(frame,(x,y),(end_coord_x,end_coord_y),(0,0,0),2)  #Drawing the rectangle on the frame

        cv2.imshow('Face Recognition',frame)

        if(flag_face_recognised):    #if face is recognized then open the door
            speak("Welcome "+name.replace('_',' ')+", unlocking door. The door will remain open for the next 5 seconds")
            print("DOOR is OPEN")
            #arduino.write(bytes('o', 'utf-8'))  #Output the given byte string over the serial port.
            
            time.sleep(5)
            speak("Closing door")
            #arduino.write(bytes('c', 'utf-8'))  #Output the given byte string over the serial port.
            print("DOOR is CLOSED")
            flag_face_recognised=False

        if(flag_face_not_recognised):
            speak("Face not recognised. The door will remain closed")    
            time.sleep(2)
            flag_face_not_recognised=False


        k=cv2.waitKey(20)     #the key is recieved and converted into ascii value
        if k==113:  #compairing with the ordinal/ascii value of 'q'
            break


    print("No. of frames : ",count_frames," |   No. of times face detected : ",total_no_face_detected)
    cap.release()
    cv2.destroyAllWindows()




if(__name__=="__main__"):
    face_recognition()    


0 86.25381494122635
PREDICTED :  jamila
0 86.40387851418646
PREDICTED :  jamila
0 87.14096678810972
PREDICTED :  jamila
0 87.31672066682208
PREDICTED :  jamila
0 86.85339428123763
PREDICTED :  jamila
0 87.36578770541433
PREDICTED :  jamila
0 85.5954385211128
PREDICTED :  jamila
0 87.45037211554848
PREDICTED :  jamila
0 86.42025038304423
PREDICTED :  jamila
0 86.42133755778698
PREDICTED :  jamila
0 84.84342308400822
PREDICTED :  jamila
0 85.59333417228228
PREDICTED :  jamila
0 85.64471300177757
PREDICTED :  jamila
0 89.03629401118833
PREDICTED :  jamila
0 86.29813558873126
PREDICTED :  jamila
0 86.56412523845823
PREDICTED :  jamila
0 86.90486054038301
PREDICTED :  jamila
DOOR is OPEN
